# Adsorption energy calculations using a pre-trained OCP calculator with OC22 referencing

For the OC22 release, we introduced total energy model predictions. This is more flexible than the adsorption energy predictions from OC20 and allows many other properties to be calculated. 

To use this model we'll have to use it separately for predictions for the gas phase species and bare slab like we did in {doc}`relaxations`. 


For this example we'll use a checkpoint for a GemNet-OC model that was trained on both OC20 and OC22 total energy data.  


`````{warning}
This checkpoint has a few important caveats to keep in mind before using it blindly:

* This is trained as a force field to predict both energies and forces for specific configurations
* The energies and forces are not internally consistent, since we've found that models that predict the energy and force separately tend to do better for OC20
`````

## Download pretrained checkpoint

We have released checkpoints of all the models on the leaderboard [here](https://github.com/Open-Catalyst-Project/ocp/blob/master/MODELS.md). These trained models can be used as an ASE calculator for various calculations.

In [ ]:
!wget -q -nc https://dl.fbaipublicfiles.com/opencatalystproject/models/2022_09/oc22/s2ef/gnoc_oc22_oc20_all_s2ef.pt
checkpoint_path = "gnoc_oc22_oc20_all_s2ef.pt"

## Relaxing the adslab

We can load the pre-trained OCP calculator very easily!

In [ ]:
from ocpmodels.common.relaxation.ase_utils import OCPCalculator

ocp_calculator = OCPCalculator(checkpoint=checkpoint_path)

Now we'll basically repeat the process from {doc}`relaxations` using our pre-trained calculator.

In [ ]:
import os

import ase.io
import numpy as np
from ase import Atoms
from ase.build import add_adsorbate, fcc100, molecule
from ase.calculators.emt import EMT
from ase.constraints import FixAtoms
from ase.io import extxyz
from ase.io.trajectory import Trajectory
from ase.optimize import LBFGS

# This cell sets up and runs a structural relaxation
# of a Cu(100) surface. It uses ASE scripts to make the surface.
# The actual surfaces in OC20 were prepared slightly differently.

# Make the bare slab using an ASE helper script
adslab = fcc100("Cu", size=(3, 3, 3))

# Now, let's add the adsorbate to the bare slab
adsorbate = molecule("CH3O")
add_adsorbate(adslab, adsorbate, 3, offset=(1, 1))  # adslab = adsorbate + slab

# Tag all slab atoms below surface as 0, surface as 1, adsorbate as 2
tags = np.zeros(len(adslab))
tags[18:27] = 1
tags[27:] = 2
adslab.set_tags(tags)

# Fixed atoms are prevented from moving during a structure relaxation.
# We fix all slab atoms beneath the surface.
cons = FixAtoms(indices=[atom.index for atom in adslab if (atom.tag == 0)])
adslab.set_constraint(cons)
adslab.center(vacuum=13.0, axis=2)
adslab.set_pbc(True)

# Set the toy calculator (EMT) so ASE knows how to get energies/forces
# at each step
adslab.set_calculator(ocp_calculator)

os.makedirs("data", exist_ok=True)

# Define structure optimizer - LBFGS. Run for 100 steps,
# or if the max force on all atoms (fmax) is below 0 ev/A.
# fmax is typically set to 0.01-0.05 eV/A,
# for this demo however we run for the full 100 steps.

dyn = LBFGS(adslab, trajectory="data/gemnetOC_oc22_Cu100+CH3O.traj")
dyn.run(fmax=0.03, steps=100)

adslab_traj = ase.io.read("data/gemnetOC_oc22_Cu100+CH3O.traj", ":")

Notice the energies are much more negative than the results in {doc}`ase_calculator_demo` since these are total DFT energies, not adsorption energies. 

Let's check to see how the relaxation went to make sure it looks reasonable.

In [ ]:
import matplotlib.pyplot as plt
from ase.visualize.plot import animate
from matplotlib import rc

# the `index` argument corresponds to what frame of the trajectory to read in, specifiying ":" reads in the full trajectory.
traj = ase.io.read("data/gemnetOC_oc22_Cu100+CH3O.traj", index=":")
anim = animate(traj, radii=0.8, rotation=("-75x, 45y, 10z"))

rc("animation", html="jshtml")
anim

Now let's predict the rest of the energies we need to compute the referenced adsorption energy. Note that the code is basically identical to the EMT example in {doc}`relaxations`; we just replace the EMT calculator with our new calculator.

We'll also use the real DFT-calculated per-element reference energies from the OC22 paper.

In [ ]:
# Adslab energy
relaxed_adslab = ase.io.read("data/gemnetOC_oc22_Cu100+CH3O.traj")
adslab_energy = relaxed_adslab.get_potential_energy()
print(f"Adsorbate+slab energy = {adslab_energy:.2f} eV")

# Corresponding raw slab used in original adslab (adsorbate+slab) system.
slab = fcc100("Cu", size=(3, 3, 3))
tags = np.zeros(len(slab))
tags[18:27] = 1
slab.set_tags(tags)
cons = FixAtoms(indices=[atom.index for atom in slab if (atom.tag == 0)])
slab.set_constraint(cons)
slab.center(vacuum=13.0, axis=2)
slab.set_calculator(ocp_calculator)
dyn = LBFGS(slab)
dyn.run(fmax=0.03, steps=100)
slab_energy = slab.get_potential_energy()
print(f"Bare slab energy = {slab_energy:.2f} eV")

# These should be fixed!
gas_reference_energies = {
    "H": 1,
    "N": 1,
    "O": 1,
    "C": 1,
}

adsorbate = Atoms("CH3O").get_chemical_symbols()

adsorbate_reference_energy = 0
for ads in adsorbate:
    adsorbate_reference_energy += gas_reference_energies[ads]

print(f"Adsorbate reference energy = {adsorbate_reference_energy:.2f} eV\n")

adsorption_energy = adslab_energy - slab_energy - adsorbate_reference_energy
print(f"Adsorption energy: {adsorption_energy:.2f} eV")

Compare the results here with what the OC20-style calculator predicted in {doc}`ase_calculator_demo`. Note that the adsorbate rotates so that the O is pointing down on the surface and the adsorbate does not associate. This looks much more reasonable!


In [ ]:
import pandas as pd
import plotly.express as px

energies = [
    image.get_potential_energy() - slab_energy - adsorbate_reference_energy
    for image in traj
]

df = pd.DataFrame(
    {
        "Relaxation Step": range(len(energies)),
        "GemNet-OC (OC20+OC22) Adsorption Energy [eV]": energies,
    }
)

fig = px.line(df, x="Relaxation Step", y="GemNet-OC (OC20+OC22) Adsorption Energy [eV]")
fig.show()